**Table of contents**<a id='toc0_'></a>    
- [简介](#toc1_)    
- [功能特点](#toc2_)    
- [安装](#toc3_)    
- [示例](#toc4_)    
  - [异步API](#toc4_1_)    
  - [指定用户目录](#toc4_2_)    
- [遇到的问题](#toc5_)    
  - [`jupyter`中不能使用同步`API`](#toc5_1_)    
    - [问题描述](#toc5_1_1_)    
    - [解决方案](#toc5_1_2_)    
  - [在`jupyter`中运行出现`NotImplementedError`错误](#toc5_2_)    
    - [问题描述](#toc5_2_1_)    
    - [解决方案](#toc5_2_2_)    
- [参考](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[简介](#toc0_)

[Playwright](https://playwright.dev/python/)是一个用于自动化网页浏览器操作的开源工具库,支持多种主流浏览器(Chrome、Firefox、Safari),用于Web应用的自动化测试、网页爬取等场景.

# <a id='toc2_'></a>[功能特点](#toc0_)

- 支持多种主流浏览器(Chrome、Firefox、Safari)
- 支持多种编程语言(JavaScript、Python、C#、Java)
- 无头(headless)模式

# <a id='toc3_'></a>[安装](#toc0_)

In [ ]:
! pip install playwright
! playwright install --with-deps

# <a id='toc4_'></a>[示例](#toc0_)

## <a id='toc4_1_'></a>[异步API](#toc0_)

In [ ]:
import asyncio
from playwright.async_api import async_playwright

    

async def main():
    async with async_playwright() as playwright:
        browser = await playwright.chromium.launch(headless=False)
        page = await browser.new_page()
        
        # 访问 GitHub 主页
        await page.goto("https://www.douyin.com/discover")
        
        # 在搜索框中输入 "playwright"
        await page.fill('input[name="q"]', "playwright")
        
        # 提交搜索
        await page.press('input[name="q"]', "Enter")
        
        # 等待搜索结果加载
        await page.wait_for_selector('ul.repo-list')
        
        # 获取第一个搜索结果的标题
        first_result = await page.locator('ul.repo-list li:first-child a:has-text("playwright")').inner_text()
        
        print(f"First search result: {first_result}")
        
        # 验证搜索结果中包含 "playwright"
        assert "playwright" in first_result.lower(), "Expected 'playwright' in search results"
        
        await browser.close()

await main()

## <a id='toc4_2_'></a>[指定用户目录](#toc0_)

有些网站需要使用验证码登录过后才能访问,这时候可以事先登录好,然后使用`playwright`基于这个已经登录的用户目录进行操作.

**1. 找到用户目录**

要找到`Chrome`的用户目录,可以在`Chrome`的地址栏输入`chrome://version/`查看`Profile Path`字段,这个字段就是用户目录的路径.

如图:

![](https://github.com/cruldra/picx-images-hosting/raw/master/image.361hoxlyqa.png)

**2. 使用用户目录运行`playwright`**


In [15]:
import asyncio
from playwright.async_api import async_playwright

    

async def main():
    async with async_playwright() as playwright:
        # 指定 Chrome 用户配置文件的路径
        user_data_dir = r"C:\Users\cruld\AppData\Local\Google\Chrome\User Data" #这里后面不要加\Default,Chrome会自动加上

        browser =await playwright.chromium.launch_persistent_context(
            user_data_dir=user_data_dir,
            channel="chrome",  # 使用安装的 Chrome 而不是 Playwright 内置的 Chromium
            headless=False  # 以有头模式运行，这样你可以看到浏览器窗口
        )
        page = await browser.new_page()
        
        # 访问 抖音 主页
        await page.goto("https://www.douyin.com/discover")
        
        # 等待直到用户关闭浏览器
        await page.wait_for_event("close",timeout=0)

await main()

# <a id='toc5_'></a>[遇到的问题](#toc0_)


## <a id='toc5_1_'></a>[`jupyter`中不能使用同步`API`](#toc0_)



### <a id='toc5_1_1_'></a>[问题描述](#toc0_)

[Error: It looks like you are using Playwright Sync API inside the asyncio loop. Please use the Async API instead. · Issue #462 · microsoft/playwright-python](https://github.com/microsoft/playwright-python/issues/462)

### <a id='toc5_1_2_'></a>[解决方案](#toc0_)

改用异步`API`



## <a id='toc5_2_'></a>[在`jupyter`中运行出现`NotImplementedError`错误](#toc0_)



### <a id='toc5_2_1_'></a>[问题描述](#toc0_)



### <a id='toc5_2_2_'></a>[解决方案](#toc0_)

找到`venv\Lib\site-packages\ipykernel\kernelapp.py`文件的第`662`行,将其注释掉

如图:

![](https://github.com/cruldra/picx-images-hosting/raw/master/image.1e8iu0f9o7.png)

**不确定会不会有其他问题,参考[这里](https://github.com/microsoft/playwright-python/issues/178#issuecomment-1302869947)**

# <a id='toc6_'></a>[参考](#toc0_)

* [Installation | Playwright Python](https://playwright.dev/python/docs/intro)